In [1]:
import libcst as cst

from libcst.display import dump

code1 = """
from Crypto.PublicKey import RSA
    
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

# Generate RSA key pair
key = RSA.generate(1024)
private_key = key
public_key = key.publickey()

# Message to sign
message = b"Hello, this is a message to sign!"

# Create a SHA256 hash of the message
hash_obj = SHA256.new(message)

# Sign the message with the private key
signature = pkcs1_15.new(private_key).sign(hash_obj)

# Verify the signature with the public key
try:
    pkcs1_15.new(public_key).verify(hash_obj, signature)
    print("Signature is valid.")
except (ValueError, TypeError):
    print("Signature is invalid.")

# Save keys and signature (optional)
with open("rsa_private.pem", "wb") as f:
    f.write(private_key.export_key())
with open("rsa_public.pem", "wb") as f:
    f.write(public_key.export_key())
with open("signature.bin", "wb") as f:
    f.write(signature)
"""

tree = cst.parse_module(code1)
print(dump(tree))

Module(
  body=[
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
            attr=Name(
              value='PublicKey',
            ),
          ),
          names=[
            ImportAlias(
              name=Name(
                value='RSA',
              ),
            ),
          ],
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
            attr=Name(
              value='Signature',
            ),
          ),
          names=[
            ImportAlias(
              name=Name(
                value='pkcs1_15',
              ),
            ),
          ],
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
       

In [2]:
from arta import RulesEngine
inputs = {
    "tree":tree
}


libname = "cryptography"

eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']

print(f"Mappings:{mappings}")

if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

mod_inputs = {
    "tree":tree,
    "mappings":mappings
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']
mod_inputs2 = {
    "tree":modified_keygen_tree,
    "mappings":mappings
}

primitive_result = eng.apply_rules(mod_inputs, rule_set="ds_primitive_rule_set")
modified_primitive_tree = primitive_result['check_primitive']


mod_inputs3 = {
    "tree":modified_primitive_tree,
    "mappings":mappings
}

result = eng.apply_rules(mod_inputs3, rule_set="ds_sign_rule_set")
modified_sign_tree = result['check_sign']


mod_inputs4 = {
    "tree":modified_sign_tree,
    "library": libname
}

verify_result = eng.apply_rules(mod_inputs4, rule_set="ds_verify_rule_set")
modified_verify_tree = verify_result['check_verify']



mod_inputs5 = {
    "tree":modified_verify_tree,
    "mappings":mappings
}

remove_result = eng.apply_rules(mod_inputs5, rule_set="remove_rule_set")
remove_pubkey_tree = remove_result['check_remove_pubkey']

mod_inputs6 = {
    "tree":remove_pubkey_tree,
    "mappings":mappings
}

add_result = eng.apply_rules(mod_inputs6, rule_set="add_rule_set")
add_hash_nodes_tree = add_result['insert_hash_nodes']


mod_inputs7 = {
    "tree":add_hash_nodes_tree,
    "mappings":mappings,
    "library":libname
}

import_result = eng.apply_rules(mod_inputs7, rule_set="update_imports_rule_set")
update_imports_tree = import_result['check_imports']



print("----OUTPUT---")
#print(modified_keygen_tree.code)
print(update_imports_tree.code)

Mappings:{'public_key': 'public_key'}
Both 'public_key' and 'private_key' must be present in mappings

----OUTPUT---

from Crypto.PublicKey import RSA
    
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

# Generate RSA key pair
key = RSA.generate(1024)
private_key = key

# Message to sign
message = b"Hello, this is a message to sign!"

# Create a SHA256 hash of the message
hash_obj = SHA256.new(message)

# Sign the message with the private key
signature = pkcs1_15.new(private_key).sign(hash_obj)

# Verify the signature with the public key
try:
    pkcs1_15.new(public_key).verify(hash_obj, signature)
    print("Signature is valid.")
except (ValueError, TypeError):
    print("Signature is invalid.")

# Save keys and signature (optional)
with open("rsa_private.pem", "wb") as f:
    f.write(private_key.export_key())
with open("rsa_public.pem", "wb") as f:
    f.write(public_key.export_key())
with open("signature.bin", "wb") as f:
    f.write(signature)



In [1]:
import libcst as cst

from libcst.display import dump

code1 = """
from nacl.signing import SigningKey

# Generate a new random signing key
signing_key = SigningKey.generate()

# Sign a message with the signing key
signed = signing_key.sign(b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = signing_key.verify_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()
"""


tree = cst.parse_module(code1)
print(dump(tree))



Module(
  body=[
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='nacl',
            ),
            attr=Name(
              value='signing',
            ),
          ),
          names=[
            ImportAlias(
              name=Name(
                value='SigningKey',
              ),
            ),
          ],
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        Assign(
          targets=[
            AssignTarget(
              target=Name(
                value='signing_key',
              ),
            ),
          ],
          value=Call(
            func=Attribute(
              value=Name(
                value='SigningKey',
              ),
              attr=Name(
                value='generate',
              ),
            ),
          ),
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        Assign(
          targets=[
            AssignTarget(
   

In [7]:
from arta import RulesEngine


inputs = {
    "tree":tree
}


libname = "cryptography"

eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']



if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

print(f"Mappings:{mappings}")


mod_inputs = {
    "tree":tree,
    "mappings":mappings
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']

mod_inputs2 = {
    "tree":modified_keygen_tree,
    "mappings":mappings
}




result = eng.apply_rules(mod_inputs2, rule_set="ds_sign_rule_set")
modified_sign_tree = result['check_sign']


print("**OUTPUT**\n")
print(modified_sign_tree.code)


['nacl.signing.SigningKey.generate'] names
['signing_key.sign'] names
['verify_key.encode'] names
Mappings:{'private_key': 'signing_key', 'message': 'b"Attack at Dawn"', 'signing_key': 'signing_key', 'public_key': 'public_signing_key'}
**OUTPUT**


from nacl.signing import SigningKey

# Generate a new random signing key
public_signing_key, signing_key = Dilithium().keygen()

# Sign a message with the signing key
signed = Dilithium().sign(signing_key, b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = signing_key.verify_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()



In [ ]:
T